In [1]:
###### i prb seq from  rb seq
from functools import reduce
import itertools
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm import trange
from multiprocessing import Pool
from tqdm import tqdm
### 2-qubit Clifford matrix
with open('Cliff_kruas.pkl', 'rb') as f:
    Cliff_kruas = pickle.load(f)
with open('rb_seq.pkl', 'rb') as f:
    rb_seq = pickle.load(f)
def gate_number(seq):
    return len(seq)-seq.count('i')-1
def find_index(_j):
    if np.isclose(abs(np.trace(interleaved_gate@Cliff_kruas[_j].conj().T))/4,1):
        return _j
def Cliff_2_primitive(_rb_seq):
    a=[]
    for _i in _rb_seq :
        if _i =='i':
            a.append('i')
        else:
            a += Cliff_index[_i]
    return a
def add_irb_inverse(_rb_seq):
    _irb_seq=list(itertools.chain(*zip(_rb_seq, [interleaved_gate_index]*len(_rb_seq))))
    _iprb_seq=list(itertools.chain(*zip(_rb_seq, ['i']*len(_rb_seq))))
    total_operation=reduce(lambda x,y: y@x, [Cliff_kruas[_i] for _i in _irb_seq])
### find irb inverse element
    for i_irb in range(11520):
        if (np.isclose(abs(np.trace(Cliff_kruas[i_irb]@total_operation)),4)):
            _iprb_seq.append(i_irb)
            
            return Cliff_2_primitive(_iprb_seq)
        elif i_irb == 11519:
            return False
if __name__ == '__main__':
    gateset=int(input('gateset \n'))
    with open('Cliff_index_gateset'+str(gateset)+'_reindex.pkl', 'rb') as f:
        Cliff_index = pickle.load(f)
    iprb_seq={}
###interleaved_ part#######################################################################################################################
#### interleaved_gate
    X =  np.asarray([[0,   1], [1,  0]])
    interleaved_gate= np.kron(X, np.eye(2))
#### find interleaved_gate index    
    pool = Pool()
    interleaved_gate_index=[i for i in filter(lambda x: x!=None, pool.map_async(find_index,[_j for _j in range(11520)]).get())][0]
    pool.close()
    pool.join()
    print('interleaved_gate_index=',interleaved_gate_index)
#######################################################################################################
    for i in tqdm(rb_seq):
        pool=Pool()
        iprb_seq[i]=pool.map(add_irb_inverse,[rb_seq[i][_j][:-1] for _j in range(len(rb_seq[i]))])
        assert False not in iprb_seq[i], 'false in irbseq'
        pool.close()
        pool.join()
####################### sorted by pluse number
    b=sorted(reduce(lambda x,y: x+y, list(iprb_seq.values())),key= lambda x: len(x)-x.count('i')-1)
    c={}
    d=[b[0]]
    for i in b[1:]:
        if gate_number(i)==gate_number(b[b.index(i)-1]):
            d.append(i)
        else:
            c[gate_number(b[b.index(i)-1])]=d
            d=[]
            d.append(i)
    with open(str(interleaved_gate_index)+'iprb_seq_gateset'+str(gateset)+'.pkl', 'wb') as f:
        pickle.dump(c, f)

KeyboardInterrupt: Interrupted by user

In [ ]:
###### i prb seq  ### random select from p
from functools import reduce
from itertools import permutations, chain
import numpy as np
import pickle
import os
from print_large import _print_big_matrix
from tqdm.notebook import tqdm
from multiprocessing import Pool
a=[1,2,3,4,5,6, 8, 10, 13, 16, 20, 25, 32, 40, 50, 63, 79, 100, 126, 158, 200, 251, 316, 398, 501, 631, 794, 1000, 1259, 1585, 1995, 2512, 3162]
def irb_seq(_iprb_seq):
    total_operation=reduce(lambda x,y: y@x, [P[_i] for _i in _iprb_seq])
### find irb inverse element
    for _i in range(len(P)):
        if (np.isclose(abs(np.trace(P[_i]@total_operation)),4)):
            _iprb_seq.append(_i)
            return _iprb_seq
        
if __name__ == '__main__':
    gateset=int(input('gateset \n'))
    interleaved_gate_index=int(input('interleaved gate number \n'))
    rep=int(input('repeat \n'))
    gateset_path=str(os.path.abspath(os.path.join(os.getcwd(), "../..")))+'/gateset/'
    if gateset==1:
        gate_distri=[1, 9601, 8129, 3103, 2296, 0, 0, 9285, 8118, 2974, 3058, 0, 0, 10523, 9788]
    elif gateset==2:
        gate_distri=[16192, 11846, 15055, 13880, 10476, 10481]
    with open(gateset_path+'gateset'+str(gateset)+'/P_kruas_gateset'+str(gateset)+'.pkl', 'rb') as f:
        P = pickle.load(f)
    with open(gateset_path+'gateset'+str(gateset)+'/Cliff_index_gateset'+str(gateset)+'_reindex.pkl', 'rb') as f:
        C_gateset = pickle.load(f)



    iprb_seq={}
    for i in tqdm(a):

###random choice#######################################################################################################
######## all possible combination        
        if i<6:
            seq= [list(_i) for _i in list(permutations([_j for _j in range(len(P))], r=i))]
######## random choices            
        else:
            np.random.seed()
            seq=np.random.choice([_i for _i in range(len(P))],size=(rep,i),p=[_i / sum(gate_distri) for _i in gate_distri]).tolist()
###flatten list
        flattenlist = lambda nestedlist:[item for element in nestedlist for item in flattenlist(element)] if type(nestedlist) is list else [nestedlist]
###interleaved gate in P
        seq=[flattenlist(list(chain(*[lst[_i:_i+1] + [C_gateset[interleaved_gate_index]] if len(lst[_i:_i+1]) == 1 else lst[_i:_i+1] for _i in range(len(lst))]))) for lst in seq]

###cal the inverse        
        pool=Pool()
#### filter None
        iprb_seq[i]=list(filter(None, pool.map_async(irb_seq,seq).get()))
        pool.close()
        pool.join()
##### end for loop #################################################################################################
    with open(str(interleaved_gate_index)+'iprb_seq_gateset'+str(gateset)+'.pkl', 'wb') as f:
        pickle.dump(iprb_seq, f)

gateset 
1
interleaved gate number 
4
repeat 
1000000


  0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
######  prb + interleaved prb  ### random select from p
from functools import reduce
from itertools import permutations, chain
import numpy as np
import pickle
import os
from print_large import _print_big_matrix
from tqdm.notebook import tqdm
from multiprocessing import Pool
a=[1,2,3,4,5,6, 8, 10, 13, 16, 20, 25, 32, 40, 50, 63, 79, 100, 126, 158, 200, 251, 316, 398, 501, 631, 794, 1000, 1259, 1585, 1995, 2512, 3162]
def irb_seq(_iprb_seq):
    total_operation=reduce(lambda x,y: y@x, [P[_i] for _i in _iprb_seq])
### find irb inverse element
    for _i in range(len(P)):
        if (np.isclose(abs(np.trace(P[_i]@total_operation)),4)):
            _iprb_seq.append(_i)
            return _iprb_seq
        
if __name__ == '__main__':
    gateset=int(input('gateset \n'))
    inter_or_not=bool(int(input('inter(1) or not(0) \n')))
    if inter_or_not== True:
        interleaved_gate_index=int(input('interleaved gate number \n'))
    rep=int(input('repeat \n'))
    gateset_path=str(os.path.abspath(os.path.join(os.getcwd(), "../..")))+'/gateset/'
    if gateset==1:
        gate_distri=[1, 9601, 8129, 3103, 2296, 0, 0, 9285, 8118, 2974, 3058, 0, 0, 10523, 9788]
    elif gateset==2:
        gate_distri=[16192, 11846, 15055, 13880, 10476, 10481]
    with open(gateset_path+'gateset'+str(gateset)+'/P_kruas_gateset'+str(gateset)+'.pkl', 'rb') as f:
        P = pickle.load(f)
    with open(gateset_path+'gateset'+str(gateset)+'/Cliff_index_gateset'+str(gateset)+'_reindex.pkl', 'rb') as f:
        C_gateset = pickle.load(f)



    iprb_seq={}
    for i in tqdm(a):

###random choice#######################################################################################################
######## all possible combination        
        if i<6:
            seq= [list(_i) for _i in list(permutations([_j for _j in range(len(P))], r=i))]
######## random choices            
        else:
            np.random.seed()
            seq=np.random.choice([_i for _i in range(len(P))],size=(rep,i),p=[_i / sum(gate_distri) for _i in gate_distri]).tolist()

        if inter_or_not== True:
###flatten list
            flattenlist = lambda nestedlist:[item for element in nestedlist for item in flattenlist(element)] if type(nestedlist) is list else [nestedlist]
###interleaved gate in P
            seq=[flattenlist(list(chain(*[lst[_i:_i+1] + [C_gateset[interleaved_gate_index]] if len(lst[_i:_i+1]) == 1 else lst[_i:_i+1] for _i in range(len(lst))]))) for lst in seq]
     
        pool=Pool()
#### cal the inverse  and filter None
        iprb_seq[i]=list(filter(None, pool.map_async(irb_seq,seq).get()))
        pool.close()
        pool.join()
##### end for loop #################################################################################################
#    with open(str(interleaved_gate_index)+'iprb_seq_gateset'+str(gateset)+'.pkl', 'wb') as f:
#        pickle.dump(iprb_seq, f)